In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/flight-price/Data_Train.xlsx
/kaggle/input/flight-price/Test_set.xlsx


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline



In [3]:
#train and test Dataset
test_df= pd.read_excel("/kaggle/input/flight-price/Test_set.xlsx")
train_df = pd.read_excel("/kaggle/input/flight-price/Data_Train.xlsx")

In [4]:
train_df.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [5]:
test_df.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info
0,Jet Airways,6/06/2019,Delhi,Cochin,DEL → BOM → COK,17:30,04:25 07 Jun,10h 55m,1 stop,No info
1,IndiGo,12/05/2019,Kolkata,Banglore,CCU → MAA → BLR,06:20,10:20,4h,1 stop,No info
2,Jet Airways,21/05/2019,Delhi,Cochin,DEL → BOM → COK,19:15,19:00 22 May,23h 45m,1 stop,In-flight meal not included
3,Multiple carriers,21/05/2019,Delhi,Cochin,DEL → BOM → COK,08:00,21:00,13h,1 stop,No info
4,Air Asia,24/06/2019,Banglore,Delhi,BLR → DEL,23:55,02:45 25 Jun,2h 50m,non-stop,No info


In [6]:
#Merging train and test Dataset
final_df = train_df.append(test_df)
final_df.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897.0
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662.0
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882.0
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218.0
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302.0


In [7]:

final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13354 entries, 0 to 2670
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Airline          13354 non-null  object 
 1   Date_of_Journey  13354 non-null  object 
 2   Source           13354 non-null  object 
 3   Destination      13354 non-null  object 
 4   Route            13353 non-null  object 
 5   Dep_Time         13354 non-null  object 
 6   Arrival_Time     13354 non-null  object 
 7   Duration         13354 non-null  object 
 8   Total_Stops      13353 non-null  object 
 9   Additional_Info  13354 non-null  object 
 10  Price            10683 non-null  float64
dtypes: float64(1), object(10)
memory usage: 1.2+ MB


In [8]:
final_df.shape

(13354, 11)

In [9]:
final_df.describe()

,Price
count,10683.000000
mean,9087.064121
std,4611.359167
min,1759.000000
25%,5277.000000
50%,8372.000000
75%,12373.000000
max,79512.000000


In [10]:
final_df["Date_of_Journey"].str.split('/')

0       [24, 03, 2019]
1        [1, 05, 2019]
2        [9, 06, 2019]
3       [12, 05, 2019]
4       [01, 03, 2019]
             ...      
2666     [6, 06, 2019]
2667    [27, 03, 2019]
2668     [6, 03, 2019]
2669     [6, 03, 2019]
2670    [15, 06, 2019]
Name: Date_of_Journey, Length: 13354, dtype: object

In [11]:
#Feature Engineering :Splitting date, Month, year from date of journey column
final_df["Date"] = final_df["Date_of_Journey"].str.split('/').str[0]
final_df["Month"] = final_df["Date_of_Journey"].str.split('/').str[1]
final_df["Year"] = final_df["Date_of_Journey"].str.split('/').str[2]


In [12]:
final_df["Date_of_Journey"].str.split('/').str[0]

0       24
1        1
2        9
3       12
4       01
        ..
2666     6
2667    27
2668     6
2669     6
2670    15
Name: Date_of_Journey, Length: 13354, dtype: object

In [13]:
final_df.head(3)

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,Date,Month,Year
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897.0,24,03,2019
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662.0,1,05,2019
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882.0,9,06,2019


In [14]:
#another technique to apply lambda function
final_df["Date"] = final_df["Date_of_Journey"].apply(lambda x:x.split('/')[0])
final_df["Month"] = final_df["Date_of_Journey"].apply(lambda x:x.split('/')[1])
final_df["Year"] = final_df["Date_of_Journey"].apply(lambda x:x.split('/')[2])

In [15]:
#converting date, month , year into int variable
final_df["Date"] = final_df["Date"].astype(int)
final_df['Month'] = final_df['Month'].astype(int)
final_df['Year']= final_df['Year'].astype(int)

In [16]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13354 entries, 0 to 2670
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Airline          13354 non-null  object 
 1   Date_of_Journey  13354 non-null  object 
 2   Source           13354 non-null  object 
 3   Destination      13354 non-null  object 
 4   Route            13353 non-null  object 
 5   Dep_Time         13354 non-null  object 
 6   Arrival_Time     13354 non-null  object 
 7   Duration         13354 non-null  object 
 8   Total_Stops      13353 non-null  object 
 9   Additional_Info  13354 non-null  object 
 10  Price            10683 non-null  float64
 11  Date             13354 non-null  int64  
 12  Month            13354 non-null  int64  
 13  Year             13354 non-null  int64  
dtypes: float64(1), int64(3), object(10)
memory usage: 1.5+ MB


In [17]:
#Removing Date of Journey feild
final_df.drop(["Date_of_Journey"], axis =1, inplace = True)


In [18]:
final_df.head()

,Airline,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,Date,Month,Year
0,IndiGo,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897.0,24,3,2019
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662.0,1,5,2019
2,Jet Airways,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882.0,9,6,2019
3,IndiGo,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218.0,12,5,2019
4,IndiGo,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302.0,1,3,2019


In [19]:
#splitting feature Arrival time 
final_df["Arrival_Time"].str.split(' ')


0       [01:10, 22, Mar]
1                [13:15]
2       [04:25, 10, Jun]
3                [23:30]
4                [21:35]
              ...       
2666    [20:25, 07, Jun]
2667             [16:55]
2668    [04:25, 07, Mar]
2669             [19:15]
2670             [19:15]
Name: Arrival_Time, Length: 13354, dtype: object

In [20]:
final_df["Arrival_Time"].str.split(' ')[0]

0    [01:10, 22, Mar]
0    [04:25, 07, Jun]
Name: Arrival_Time, dtype: object

In [21]:

#Applying Lambda function for splitting on Arrival time
final_df["Arrival_Time"] = final_df["Arrival_Time"].apply(lambda x: x.split(' ')[0])

In [22]:
final_df.isnull().sum()


Airline               0
Source                0
Destination           0
Route                 1
Dep_Time              0
Arrival_Time          0
Duration              0
Total_Stops           1
Additional_Info       0
Price              2671
Date                  0
Month                 0
Year                  0
dtype: int64

In [23]:
#lambda function for splitting on Arrival time
final_df["Arrival_hour"] = final_df["Arrival_Time"].str.split(':').str[0]
final_df["Arrival_min"] = final_df["Arrival_Time"].str.split(':').str[1]

In [24]:
final_df.head()

,Airline,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,Date,Month,Year,Arrival_hour,Arrival_min
0,IndiGo,Banglore,New Delhi,BLR → DEL,22:20,01:10,2h 50m,non-stop,No info,3897.0,24,3,2019,01,10
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662.0,1,5,2019,13,15
2,Jet Airways,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25,19h,2 stops,No info,13882.0,9,6,2019,04,25
3,IndiGo,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218.0,12,5,2019,23,30
4,IndiGo,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302.0,1,3,2019,21,35


In [25]:
#converting hour, minute into int variable
final_df["Arrival_hour"] = final_df["Arrival_hour"].astype(int)
final_df["Arrival_min"] = final_df["Arrival_min"].astype(int)

In [26]:
#dropping arrival time after splitting and feature engineering
final_df.drop(["Arrival_Time"],axis = 1, inplace =True)

In [27]:
final_df.head(5)

,Airline,Source,Destination,Route,Dep_Time,Duration,Total_Stops,Additional_Info,Price,Date,Month,Year,Arrival_hour,Arrival_min
0,IndiGo,Banglore,New Delhi,BLR → DEL,22:20,2h 50m,non-stop,No info,3897.0,24,3,2019,1,10
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,7h 25m,2 stops,No info,7662.0,1,5,2019,13,15
2,Jet Airways,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,19h,2 stops,No info,13882.0,9,6,2019,4,25
3,IndiGo,Kolkata,Banglore,CCU → NAG → BLR,18:05,5h 25m,1 stop,No info,6218.0,12,5,2019,23,30
4,IndiGo,Banglore,New Delhi,BLR → NAG → DEL,16:50,4h 45m,1 stop,No info,13302.0,1,3,2019,21,35


In [28]:
#Splitting departure hour , minute and converting into int variable
final_df["Dep_hour"] = final_df["Dep_Time"].str.split(':').str[0]
final_df["Dep_min"] = final_df["Dep_Time"].str.split(':').str[1]

final_df["Dep_hour"] = final_df["Dep_hour"].astype(int)
final_df["Dep_min"] = final_df["Dep_min"].astype(int)

final_df.drop(["Dep_Time"],axis = 1, inplace =True)

In [29]:
final_df.head()

,Airline,Source,Destination,Route,Duration,Total_Stops,Additional_Info,Price,Date,Month,Year,Arrival_hour,Arrival_min,Dep_hour,Dep_min
0,IndiGo,Banglore,New Delhi,BLR → DEL,2h 50m,non-stop,No info,3897.0,24,3,2019,1,10,22,20
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,7h 25m,2 stops,No info,7662.0,1,5,2019,13,15,5,50
2,Jet Airways,Delhi,Cochin,DEL → LKO → BOM → COK,19h,2 stops,No info,13882.0,9,6,2019,4,25,9,25
3,IndiGo,Kolkata,Banglore,CCU → NAG → BLR,5h 25m,1 stop,No info,6218.0,12,5,2019,23,30,18,5
4,IndiGo,Banglore,New Delhi,BLR → NAG → DEL,4h 45m,1 stop,No info,13302.0,1,3,2019,21,35,16,50


In [30]:
 #unique values detection
    final_df["Total_Stops"].unique()

array(['non-stop', '2 stops', '1 stop', '3 stops', nan, '4 stops'],
      dtype=object)

In [31]:
#filling nan values with mode
final_df['Total_Stops'].fillna(final_df['Total_Stops'].mode()[0], inplace=True)

In [32]:
#mapping numerical values to stops on total stops from categorical
final_df["Total_Stops"].map({"non-stop":0, "1 stop": 1, "2 stops": 2, "3stops": 3,"4 stops": 4})

0       0.0
1       2.0
2       2.0
3       1.0
4       1.0
       ... 
2666    1.0
2667    0.0
2668    1.0
2669    1.0
2670    1.0
Name: Total_Stops, Length: 13354, dtype: float64

In [33]:
final_df.head(2)


,Airline,Source,Destination,Route,Duration,Total_Stops,Additional_Info,Price,Date,Month,Year,Arrival_hour,Arrival_min,Dep_hour,Dep_min
0,IndiGo,Banglore,New Delhi,BLR → DEL,2h 50m,non-stop,No info,3897.0,24,3,2019,1,10,22,20
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,7h 25m,2 stops,No info,7662.0,1,5,2019,13,15,5,50


In [34]:
#Dropping route feature
final_df.drop("Route",axis = 1,inplace= True)

In [35]:
final_df.head(2)

,Airline,Source,Destination,Duration,Total_Stops,Additional_Info,Price,Date,Month,Year,Arrival_hour,Arrival_min,Dep_hour,Dep_min
0,IndiGo,Banglore,New Delhi,2h 50m,non-stop,No info,3897.0,24,3,2019,1,10,22,20
1,Air India,Kolkata,Banglore,7h 25m,2 stops,No info,7662.0,1,5,2019,13,15,5,50


In [36]:
#finding unique values to Additional info column
final_df["Additional_Info"].unique()

array(['No info', 'In-flight meal not included',
       'No check-in baggage included', '1 Short layover', 'No Info',
       '1 Long layover', 'Change airports', 'Business class',
       'Red-eye flight', '2 Long layover'], dtype=object)

In [37]:
final_df.shape

(13354, 14)

In [38]:
#Splitting Duration Feature
final_df["Duration"].str.split(" ").str[0]

0        2h
1        7h
2       19h
3        5h
4        4h
       ... 
2666    23h
2667     2h
2668     6h
2669    15h
2670    14h
Name: Duration, Length: 13354, dtype: object

In [39]:
#Seperating hour from duration feature(Removing "h")
final_df["Duration"].str.split(" ").str[0].str.split("h").str[0]

0        2
1        7
2       19
3        5
4        4
        ..
2666    23
2667     2
2668     6
2669    15
2670    14
Name: Duration, Length: 13354, dtype: object

In [40]:
#splitting duration hour from duration column
final_df["Duration_hour"] = final_df["Duration"].str.split(" ").str[0].str.split("h").str[0]
final_df.head(2)

,Airline,Source,Destination,Duration,Total_Stops,Additional_Info,Price,Date,Month,Year,Arrival_hour,Arrival_min,Dep_hour,Dep_min,Duration_hour
0,IndiGo,Banglore,New Delhi,2h 50m,non-stop,No info,3897.0,24,3,2019,1,10,22,20,2
1,Air India,Kolkata,Banglore,7h 25m,2 stops,No info,7662.0,1,5,2019,13,15,5,50,7


In [41]:
## 2 records showing 5 min duration
final_df[final_df["Duration_hour"]== "5m"]

,Airline,Source,Destination,Duration,Total_Stops,Additional_Info,Price,Date,Month,Year,Arrival_hour,Arrival_min,Dep_hour,Dep_min,Duration_hour
6474,Air India,Mumbai,Hyderabad,5m,2 stops,No info,17327.0,6,3,2019,16,55,16,50,5m
2660,Air India,Mumbai,Hyderabad,5m,2 stops,No info,NaN,12,3,2019,16,55,16,50,5m


In [42]:
#need to drop these 2 records as they have 5m as duration 
#dropping Record 1
final_df.drop(6474,axis=0, inplace=True)


In [43]:
#Dropping Record 2
final_df.drop(2660,axis=0, inplace=True)

In [44]:
#dropped both records showing empty table now
final_df[final_df["Duration_hour"]== "5m"]

,Airline,Source,Destination,Duration,Total_Stops,Additional_Info,Price,Date,Month,Year,Arrival_hour,Arrival_min,Dep_hour,Dep_min,Duration_hour


In [45]:
final_df["Duration_hour"] = final_df["Duration_hour"].astype(int)

In [46]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13351 entries, 0 to 2670
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Airline          13351 non-null  object 
 1   Source           13351 non-null  object 
 2   Destination      13351 non-null  object 
 3   Duration         13351 non-null  object 
 4   Total_Stops      13351 non-null  object 
 5   Additional_Info  13351 non-null  object 
 6   Price            10681 non-null  float64
 7   Date             13351 non-null  int64  
 8   Month            13351 non-null  int64  
 9   Year             13351 non-null  int64  
 10  Arrival_hour     13351 non-null  int64  
 11  Arrival_min      13351 non-null  int64  
 12  Dep_hour         13351 non-null  int64  
 13  Dep_min          13351 non-null  int64  
 14  Duration_hour    13351 non-null  int64  
dtypes: float64(1), int64(8), object(6)
memory usage: 1.6+ MB


In [47]:
#drop Duration feature
final_df.drop(["Duration"],axis =1,inplace=True)

In [48]:
final_df.head(2)

,Airline,Source,Destination,Total_Stops,Additional_Info,Price,Date,Month,Year,Arrival_hour,Arrival_min,Dep_hour,Dep_min,Duration_hour
0,IndiGo,Banglore,New Delhi,non-stop,No info,3897.0,24,3,2019,1,10,22,20,2
1,Air India,Kolkata,Banglore,2 stops,No info,7662.0,1,5,2019,13,15,5,50,7


In [49]:
#Finding unique values on column Airline
final_df["Airline"].unique()

array(['IndiGo', 'Air India', 'Jet Airways', 'SpiceJet',
       'Multiple carriers', 'GoAir', 'Vistara', 'Air Asia',
       'Vistara Premium economy', 'Jet Airways Business',
       'Multiple carriers Premium economy', 'Trujet'], dtype=object)

In [50]:
#importing Label encoder for remaining features as Airline, Source, destination, Additional info
# Import label encoder
from sklearn import preprocessing
  
# label_encoder object knows how to understand word labels.
label_encoder = preprocessing.LabelEncoder()
  
# Encode labels in column 'species'.
final_df['Airline']= label_encoder.fit_transform(final_df['Airline'])
  
final_df['Airline'].unique()

array([ 3,  1,  4,  8,  6,  2, 10,  0, 11,  5,  7,  9])

In [51]:
final_df['Source']= label_encoder.fit_transform(final_df['Source'])
final_df['Destination']= label_encoder.fit_transform(final_df['Destination'])
final_df['Additional_Info']= label_encoder.fit_transform(final_df['Additional_Info'])

In [52]:
final_df.shape

(13351, 14)

In [53]:
final_df.head()

,Airline,Source,Destination,Total_Stops,Additional_Info,Price,Date,Month,Year,Arrival_hour,Arrival_min,Dep_hour,Dep_min,Duration_hour
0,3,0,5,non-stop,8,3897.0,24,3,2019,1,10,22,20,2
1,1,3,0,2 stops,8,7662.0,1,5,2019,13,15,5,50,7
2,4,2,1,2 stops,8,13882.0,9,6,2019,4,25,9,25,19
3,3,3,0,1 stop,8,6218.0,12,5,2019,23,30,18,5,5
4,3,0,5,1 stop,8,13302.0,1,3,2019,21,35,16,50,4


In [54]:
final_df["Airline"]


0       3
1       1
2       4
3       3
4       3
       ..
2666    1
2667    3
2668    4
2669    1
2670    6
Name: Airline, Length: 13351, dtype: int64

In [57]:
from sklearn.preprocessing import OneHotEncoder
#Create an instance of One-hot-encoder
ohe=OneHotEncoder()

In [58]:
ohe.fit_transform(np.array(final_df['Airline']).reshape(-1,1))

<13351x12 sparse matrix of type '<class 'numpy.float64'>'
	with 13351 stored elements in Compressed Sparse Row format>

In [ ]:
#one hot Encoding is used to convert text or categorical feature into numerical feature. likw "0" n "1"
#binning is used to convert numerical to categorical 
#label encoder is used to ctegorical to numerical values (multiple)

In [59]:
final_df.head()


,Airline,Source,Destination,Total_Stops,Additional_Info,Price,Date,Month,Year,Arrival_hour,Arrival_min,Dep_hour,Dep_min,Duration_hour
0,3,0,5,non-stop,8,3897.0,24,3,2019,1,10,22,20,2
1,1,3,0,2 stops,8,7662.0,1,5,2019,13,15,5,50,7
2,4,2,1,2 stops,8,13882.0,9,6,2019,4,25,9,25,19
3,3,3,0,1 stop,8,6218.0,12,5,2019,23,30,18,5,5
4,3,0,5,1 stop,8,13302.0,1,3,2019,21,35,16,50,4
